AutoKeras is an open-source library for performing AutoML for deep learning models. The search is performed using so-called Keras models via the TensorFlow tf.keras API.

It provides a simple and effective approach for automatically finding top-performing models for a wide range of predictive modeling tasks, including tabular or so-called structured classification and regression datasets.

It is one of the AutoML frameworks. AutoML refers to techniques for automatically discovering the best-performing model for a given dataset.

When applied to neural networks, this involves both discovering the model architecture and the hyperparameters used to train the model, generally referred to as neural architecture search.

Automated Machine Learning, or AutoML for short, refers to automatically finding the best combination of data preparation, model, and model hyperparameters for a predictive modeling problem.

The benefit of AutoML is allowing machine learning practitioners to quickly and effectively address predictive modeling tasks with very little input.



Installing autokeras

In [1]:
pip install autokeras

     |████████████████████████████████| 162 kB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 45.6 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 43.4 MB/s 


### 1. Classification with AutoKeras

AutoKeras can be used to discover a good or great model for classification tasks on tabular data.

Tabular data are those datasets composed of rows and columns, such as a table or data as you would see in a spreadsheet.

We will develop a model for the Sonar classification dataset for classifying sonar returns as rocks or mines. This dataset consists of 208 rows of data with 60 input features and a target class label of 0 (rock) or 1 (mine).

A naive model can achieve a classification accuracy of about 53.4 percent via repeated 10-fold cross-validation, which provides a lower-bound. A good model can achieve an accuracy of about 88.2 percent, providing an upper-bound.

In [2]:
# use autokeras to find a model for the sonar dataset
from numpy import asarray
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from autokeras import StructuredDataClassifier

In [3]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv'
dataframe = read_csv(url, header=None)
print(dataframe.shape)

(208, 61)


In [4]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [5]:
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
# basic data preparation
X = X.astype('float32')
y = LabelEncoder().fit_transform(y)

(208, 60) (208,)


In [6]:
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(139, 60) (69, 60) (139,) (69,)


In [7]:
# define the search
search = StructuredDataClassifier(max_trials=15)
# perform the search
search.fit(x=X_train, y=y_train, verbose=0)

INFO:tensorflow:Oracle triggered exit
5/5 [==============================] - 1s 5ms/step - loss: 0.8994 - accuracy: 0.4460
INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [8]:
# evaluate the model
loss, acc = search.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f' % acc)

Accuracy: 0.507


In [9]:
# use the model to make a prediction
row = [0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032]
X_new = asarray([row]).astype('float32')
yhat = search.predict(X_new)
print('Predicted: %.3f' % yhat[0])

1/1 [==============================] - 0s 15ms/step
Predicted: 1.000


In [10]:
# get the best performing model
model = search.export_model()

In [11]:
# summarize the loaded model
model.summary()
# save the best performing model to file
model.save('model_sonar.h5')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 60)               0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 60)               121       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                1952      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

We can see the best model that the autokeras has determined above. With the different layers and parameter configuration.

2. Regression models with AutoKeras

AutoKeras can also be used for regression tasks, that is, predictive modeling problems where a numeric value is predicted.

We will use the auto insurance dataset that involves predicting the total payment from claims given the total number of claims. The dataset has 63 rows and one input and one output variable.

A naive model can achieve a mean absolute error (MAE) of about 66 using repeated 10-fold cross-validation, providing a lower-bound on expected performance. A good model can achieve a MAE of about 28, providing a performance upper-bound.

In [12]:
# use autokeras to find a model for the insurance dataset
from numpy import asarray
from pandas import read_csv
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataRegressor

In [13]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/auto-insurance.csv'
dataframe = read_csv(url, header=None)
print(dataframe.shape)

(63, 2)


In [14]:
# split into input and output elements
data = dataframe.values
data = data.astype('float32')
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

(63, 1) (63,)


In [15]:
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(42, 1) (21, 1) (42,) (21,)


In [16]:
# define the search
search = StructuredDataRegressor(max_trials=15, loss='mean_absolute_error')
# perform the search
search.fit(x=X_train, y=y_train, verbose=0)


INFO:tensorflow:Oracle triggered exit
2/2 [==============================] - 0s 8ms/step - loss: 86.3103 - mean_squared_error: 13209.6768
INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [17]:
# evaluate the model
mae, _ = search.evaluate(X_test, y_test, verbose=0)
print('MAE: %.3f' % mae)


MAE: 115.306


In [18]:
# use the model to make a prediction
X_new = asarray([[108]]).astype('float32')
yhat = search.predict(X_new)
print('Predicted: %.3f' % yhat[0])


1/1 [==============================] - 0s 13ms/step
Predicted: 9.980


In [19]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 1)                0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                64        
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 re_lu_1 (ReLU)              (None, 64)                0         
                                                             

We can see the best model that the autokeras has determined above. With the different layers and parameter configuration.

In [20]:
# save the best performing model to file
model.save('model_insurance.h5')

### 3. Multimodal models with AutoKeras

Modality refers to the way in which something happens or is experienced and a research problem is characterized as multimodal when it includes multiple such modalities. In order for Artificial Intelligence to make progress in understanding the world around us, it needs to be able to interpret such multimodal signals together.

Multimodal data similarly is data which has multiple input types and we want to get one output from them. Think of this as this, there is a image dataset but we also have text descriptions for these images. So this model will take both the image as well as the text data and try to give output based on that.

In this example we try to create our own multimodal data. We do this for the sake of this example as using an actual dataset would take a lot of processing time which is beyond the scope of this submission.

In [21]:
import numpy as np

num_instances = 100
# Generate image data.
image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
# Generate structured data.
structured_data = np.random.rand(num_instances, 20).astype(np.float32)

In [22]:
# Generate regression targets.
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
# Generate classification labels of five classes.
classification_target = np.random.randint(5, size=num_instances)

In [23]:
import autokeras

# Initialize the multi with multiple inputs and outputs.
model = autokeras.AutoModel(
    inputs=[autokeras.ImageInput(), autokeras.StructuredDataInput()],
    outputs=[
        autokeras.RegressionHead(metrics=["mae"]),
        autokeras.ClassificationHead(loss="categorical_crossentropy", metrics=["accuracy"]),
    ],
    overwrite=True,
    max_trials=2,
)

In [24]:

model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=2,
)

Trial 2 Complete [00h 00m 03s]
val_loss: 1.7416462898254395

Best val_loss So Far: 1.7416462898254395
Total elapsed time: 00h 00m 34s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
4/4 [==============================] - 1s 65ms/step - loss: 1.7875 - regression_head_1_loss: 0.1393 - classification_head_1_loss: 1.6482 - regression_head_1_mae: 0.3009 - classification_head_1_accuracy: 0.1300
Epoch 2/2
4/4 [==============================] - 0s 66ms/step - loss: 1.7195 - regression_head_1_loss: 0.1092 - classification_head_1_loss: 1.6102 - regression_head_1_mae: 0.2772 - classification_head_1_accuracy: 0.1700
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [25]:
input_node1 = autokeras.ImageInput()
output_node = autokeras.Normalization()(input_node1)
output_node = autokeras.ImageAugmentation()(output_node)
output_node1 = autokeras.ConvBlock()(output_node)
output_node2 = autokeras.ResNetBlock(version="v2")(output_node)
output_node1 = autokeras.Merge()([output_node1, output_node2])

input_node2 = autokeras.StructuredDataInput()
output_node = autokeras.CategoricalToNumerical()(input_node2)
output_node2 = autokeras.DenseBlock()(output_node)

output_node = autokeras.Merge()([output_node1, output_node2])
output_node1 = autokeras.ClassificationHead()(output_node)
output_node2 = autokeras.RegressionHead()(output_node)

auto_model = autokeras.AutoModel(
    inputs=[input_node1, input_node2],
    outputs=[output_node1, output_node2],
    overwrite=True,
    max_trials=2,
)

image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
structured_data = np.random.rand(num_instances, 20).astype(np.float32)
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
classification_target = np.random.randint(5, size=num_instances)

auto_model.fit(
    [image_data, structured_data],
    [classification_target, regression_target],
    batch_size=32,
    epochs=3,
)

Trial 2 Complete [00h 00m 36s]
val_loss: 1.8255788087844849

Best val_loss So Far: 1.7687197923660278
Total elapsed time: 00h 01m 07s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
4/4 [==============================] - 14s 1s/step - loss: 3.3443 - classification_head_1_loss: 1.9439 - regression_head_1_loss: 1.4004 - classification_head_1_accuracy: 0.1900 - regression_head_1_mean_squared_error: 1.4004
Epoch 2/3
4/4 [==============================] - 7s 1s/step - loss: 4.0670 - classification_head_1_loss: 1.9664 - regression_head_1_loss: 2.1006 - classification_head_1_accuracy: 0.2000 - regression_head_1_mean_squared_error: 2.1006
Epoch 3/3
4/4 [==============================] - 7s 2s/step - loss: 4.1852 - classification_head_1_loss: 1.8029 - regression_head_1_loss: 2.3823 - classification_head_1_accuracy: 0.2000 - regression_head_1_mean_squared_error: 2.3823
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
